#**INICIALIZADORES**

-----------------------------

In [ ]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [ ]:
ee.Initialize()

#**AREA ESTUDIO**

In [ ]:
#@title AREA ESTUDIO
areae ={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -78.7060546875,
              -4.171115454867424
            ],
            [
              -67.19238281249999,
              -4.171115454867424
            ],
            [
              -67.19238281249999,
              11.781325296112277
            ],
            [
              -78.7060546875,
              11.781325296112277
            ],
            [
              -78.7060546875,
              -4.171115454867424
            ]
          ]
        ]
      }
    }
  ]
}
ee_areae = ee.Feature(areae['features'][0])
#SI VAS A USAR UNA GEOMETRIA SHAPE QUE SUBISTE PONLA AQUI ABAJO
ee_areae = ee.FeatureCollection("users/lllluisgh/AMVA")

centroide = ee_areae.geometry().centroid().getInfo()['coordinates']

# **VIZUALIZA TU AREA**

In [ ]:
#@title VIZUALIZA
# Displaye los resultados
dicc = {'area_de_estudio':ee_areae.getInfo()}
mapdisplay(centroide, dicc, zoom_start= 11)

# **FUNCION DE ENMASCARAMIETO Y ALMACENAMIENTO**

In [ ]:
#@title 1) Enmascara y filtra, 2) Mosaico
def limpio_s2(init_fecha, last_fecha, geometria, nubes_per, meses, bandas):
  def maskS2clouds(image):
    # aqui selecciona la banda de calidad
    qa = image.select('QA60')
    #aqui selecciona nubes por el valor que tiene los pixeles de esa banda
    opaque_cloud = 1 << 10
    cirrus_cloud = 1 << 11
    # va a seleccionar los pixeles y le dara el valor de cero y crea una mascara
    mask = qa.bitwiseAnd(opaque_cloud).eq(0)\
             .And(qa.bitwiseAnd(cirrus_cloud).eq(0))
    # aqui va a hacer enmascaramiento usando la capa creada anteriormente  y ademas solo seleccionara las bandas de interes
    imagen_limpia = image.updateMask(mask).select(bandas)
    return imagen_limpia.copyProperties(image, ['system:time_start'])
  #con esta funcion  calculamos calculamos el ndvi y se lo adicionamos como banda a la imagen
  def addNDVI(image):
    ndvi1 = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    ndwi = image.normalizedDifference(['B3','B8']).rename('ndwi')
    return image.addBands([ndvi1],[ndwi])
  coleccion = ee.ImageCollection("COPERNICUS/S2_SR")\
               .filterBounds(geometria)\
               .filterDate(init_fecha, last_fecha)\
               .filter(ee.Filter.calendarRange(meses[0], meses[1],'month'))\
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', nubes_per))\
               .map(maskS2clouds)#\
               #.map(addNDVI)
               #TODAS LAS IMAGENES LE HALLA EL NDVI, TU VERAS SI LO DESCARGAS O NO

  return coleccion
#---------------------------------------------------------------------------------------------
def mozzai_s2(init_fecha, last_fecha, geometria, nubes_per, meses, bandas):
  def maskS2clouds(image):
    # aqui selecciona la banda de calidad
    qa = image.select('QA60')
    #aqui selecciona nubes por el valor que tiene los pixeles de esa banda
    opaque_cloud = 1 << 10
    cirrus_cloud = 1 << 11
    # va a seleccionar los pixeles y le dara el valor de cero y crea una mascara
    mask = qa.bitwiseAnd(opaque_cloud).eq(0)\
             .And(qa.bitwiseAnd(cirrus_cloud).eq(0))
    # aqui va a hacer enmascaramiento usando la capa creada anteriormente  y ademas solo seleccionara las bandas de interes
    imagen_limpia = image.updateMask(mask).select(bandas)

    return imagen_limpia.copyProperties(image, ['system:time_start'])

  coleccion = ee.ImageCollection("COPERNICUS/S2_SR")\
               .filterBounds(geometria)\
               .filterDate(init_fecha, last_fecha)\
               .filter(ee.Filter.calendarRange(meses[0], meses[1],'month'))\
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', nubes_per))\
               .map(maskS2clouds)\
               .mosaic()\
               .clip(ee_areae)



  #NDVI = coleccion.normalizedDifference(['B8', 'B4']).rename('NDVI')
  #NDWI = coleccion.normalizedDifference(['B3', 'B8']).rename('NDWI')
  #coleccion = coleccion.addBands([NDVI])
  #coleccion = coleccion.addBands([NDWI])
               #si vas  a hacer mosaico cambia abajo todo lo que tiene ver con informacion de las imagenes
  return coleccion


#**LLAMADORES DE LA FUNCION ANTERIOR**

In [ ]:

#@title llamador para varias imagenes SI SOLO QUIERES DESCARGAR IMEGENES BUENAS
s2 = limpio_s2(init_fecha = '2019-1-1',
              last_fecha = '2021-8-28',
              geometria = ee_areae.geometry(),
              nubes_per = 2,
              meses =  [0,12],
              bandas =  ['B11','B8','B4','B3','B2'])
ic = s2

#el_ndvi = s2.normalizedDifference(['B8','B4'])

s2.size().getInfo()

9

In [ ]:
#@title llamador para EL MOSAICO -SOLO TE TIRA EL MOSAICO
s2m = mozzai_s2(init_fecha = '2019-10-1',
               last_fecha = '2021-12-28',
               geometria = ee_areae.geometry(),
               nubes_per = 2,
               meses =  [0,12],
               bandas =  ['B12','B11','B8','B4','B3','B2'])
ic = s2m
el_ndvi = s2m.normalizedDifference(['B8','B4'])
el_ndwi = s2m.normalizedDifference(['B4','B8'])

#**FECHAS DEL CONJUNTO DE IMAGENES**

In [ ]:
#@title Obtener fechas
from datetime import datetime as dt
def get_dates(s2):
  dates = s2.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: dt.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S')
  return list(map(anonym, dates))
get_dates(s2)

['2019-01-02 15:41:25',
 '2019-01-02 15:41:11',
 '2019-07-06 15:41:32',
 '2019-07-18 15:31:27',
 '2019-08-27 15:31:38',
 '2019-08-27 15:31:23',
 '2020-01-07 15:41:09',
 '2020-02-18 15:31:27',
 '2020-02-18 15:31:13']

#**Monitoreo**

In [ ]:
#@title Monitoreo
from ee.batch import Export
from datetime import datetime
import time

def ee_monitoring(ee_task):
  while ee_task.active():
    print('Sondeo de la tarea (id: {}).'.format(ee_task.id))
    time.sleep(10)

#**DESCARGAR CONJUNTO DE IMAGENES Y MOSAICO**

In [ ]:
#@title FUNCION descargar conjunto de datos
def ee_download_images(ic, region, prefix, folder, scale):
  nimages = ic.size().getInfo()
  ic_list = ic.toList(nimages)
  for n_img in range(nimages):
    to_download = ee.Image(ic_list.get(n_img))
    timestamp = to_download.get('system:time_start').getInfo()
    dt_object = datetime.fromtimestamp(timestamp/1000).strftime('%Y_%m_%d_%H_%M_%S')
    task = Export.image.toDrive(
      image=to_download,
      description= prefix+dt_object,
      folder=folder,
      scale=scale,
      region = region
    )
    print('Descargando: '+prefix+dt_object+'.tif')
    task.start()
    ee_monitoring(task)

In [ ]:
#@title FUNCION descarga mosaico
def ee_download_image_m(ic, region, prefix, folder, scale):
  #nimages = ic.size().getInfo()
  #ic_list = ic.toList(nimages)
  #for n_img in range(nimages):
    #to_download = ee.Image(ic_list.get(n_img))
    #timestamp = to_download.get('system:time_start').getInfo()
    #dt_object = datetime.fromtimestamp(timestamp/1000).strftime('%Y_%m_%d_%H_%M_%S')
    task = Export.image.toDrive(
      image=ic,
      description= prefix,
      folder=folder,
      scale=scale,
      region = region
    )
    print('Descargando: '+prefix+'.tif')
    task.start()
    ee_monitoring(task)

# **Visualizador**

In [ ]:
ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00", "#000000"]
ndwi_palette = ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
ndvi2_palette = [
    "0602ff","235cb1","307ef3","269db1","30c8e2","32d3ef",
    "3be285","3ff38f","86e26f","3ae237","b5e22e","d6e21f",
    "fff705","ffd611","ffb613","ff8b13","ff6e08","ff500d",
    "ff0000","de0101","c21301","a71001","911003"
  ]
C_viz = {
         'NDWI': el_ndwi.getMapId({'min':-0.9,'max':0.2,'palette':ndvi2_palette}),\
         'NDVI': el_ndvi.getMapId({'min':-0.2,'max':0.8,'palette':ndvi_palette}),\
         'RGB':s2m.getMapId({'bands':['B4','B3','B2'],'max':6000,'min:':0,'gamma':1.5 }),\
         }
mapdisplay(centroide,C_viz, zoom_start= 12)

#'area_de_estudio':ee_areae.getInfo(),\

# **LLAMADOR A DESCARGA**

In [ ]:
#@title Descarga conjunto de imagenes
ee_download_images(ic = s2,
                   region =  ee_areae.geometry(),
                   prefix =  '11',
                   folder = 'VEGACHI',
                   scale =  10)

In [ ]:
#@title Descarga MOSAICO
ee_download_image_m(ic = s2m,
                   region =  ee_areae.geometry(),
                   prefix =  'ENERO20_12',
                   folder = 'LORICA',
                   scale =  10)

Descargando: ENERO20_12.tif
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).
Sondeo de la tarea (id: M2ONMJWTRJVN5SLFJCCRWXYF).


In [ ]:

#@title JUNTA TODO
def composite_l7l8_vegetation(init_date, last_date, roi, cloud_per, calendar, composite = 'median'):
  '''
  Crear un composite de vegetacion a partir de datos Landsat 8 y Landsat  7

  init_date: Fecha de inicio
  last_date: Fecha de fin
  roi: Ambito de estudio
  cloud_per: Porcentaje de nubes
  calendar: Lista numerica de dos elementos. Que indica los meses a filtrar.
  composite: Funcion utilizada para realizar el composite. Actualmente
             disponible: mean, ndvi (max) y none (no composite).
  '''
  def l7_maskcloud(image):
    qa = image.select('pixel_qa')
    my_mask = (1 << 3) + (1 << 5) + (1 << 7)
    mask = qa.bitwiseAnd(my_mask).eq(0)
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)\
                .select(['B7','B4','B3','B2','B1','NDVI'])\
                .rename('SWIR2','NIR','RED','GREEN','BLUE','NDVI')\
                .updateMask(mask)\
                .copyProperties(image, ["system:time_start"])

  l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
        .filterBounds(roi)\
        .filterDate(init_date, last_date)\
        .filterMetadata('CLOUD_COVER','less_than',cloud_per)\
        .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
        .map(l7_maskcloud)

  l45 =l5
  if composite == 'median':
    return l45.median().clip(ee_areae)
  elif composite == 'ndvi':
    return l45.qualityMosaic('NDVI').clip(ee_areae)
  else:
    return l45
#@title JUNTA TODO
def composite_l8_vegetation(init_date, last_date, roi, cloud_per, calendar, composite = 'median'):
  '''
  Crear un composite de vegetacion a partir de datos Landsat 8 y Landsat  7

  init_date: Fecha de inicio
  last_date: Fecha de fin
  roi: Ambito de estudio
  cloud_per: Porcentaje de nubes
  calendar: Lista numerica de dos elementos. Que indica los meses a filtrar.
  composite: Funcion utilizada para realizar el composite. Actualmente
             disponible: mean, ndvi (max) y none (no composite).
  '''
  def l7_maskcloud(image):
    qa = image.select('pixel_qa')
    my_mask = (1 << 3) + (1 << 5) + (1 << 7) + (1 << 9) + (1 << 10)
    mask = qa.bitwiseAnd(my_mask).eq(0)
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)\
                .select(['B5','B4','B3','B2','NDVI'])\
                .rename('NIR','RED','GREEN','BLUE','NDVI')\
                .updateMask(mask)\
                .copyProperties(image, ["system:time_start"])

  l5 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterBounds(roi)\
        .filterDate(init_date, last_date)\
        .filterMetadata('CLOUD_COVER','less_than',cloud_per)\
        .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
        .map(l7_maskcloud)

  l45 =l5
  if composite == 'median':
    return l45.median().clip(ee_areae)
  elif composite == 'ndvi':
    return l45.qualityMosaic('NDVI').clip(ee_areae)
  else:
    return l45

In [ ]:
#@title CORRE LA FUNCION ANTERIOR
# ------------------------------
#  PARAMETROS GLOBALES
# ------------------------------
range_date = ['2020-01-01', '2021-12-31'] #fecha de inicio y fin
center = ee_areae.geometry().centroid().getInfo()['coordinates']# [-72.628,-10.609]
roi = ee_areae.geometry() #Area de interes (seleccione el path and row)
calendar = [0,12] # Seleccionar imagenes de Abril a Noviembre
cloud_per = 100# Maximo porcentaje de cobertura de nubes a considerar
composite = 'ndvi' # Funcion para generar el composite (median, ndvi)


compos = composite_l7l8_vegetation(init_date=range_date[0],
                                      last_date=range_date[1],
                                      roi=roi,
                                      cloud_per=cloud_per,
                                      calendar=calendar,
                                      composite='ndvi')


#-------------------------------------------------------------------------------------------------
range_date = ['2006-01-01', '2006-12-31'] #fecha de inicio y fin
center = ee_areae.geometry().centroid().getInfo()['coordinates']# [-72.628,-10.609]
roi = ee_areae.geometry() #Area de interes (seleccione el path and row)
calendar = [0,12] # Seleccionar imagenes de Abril a Noviembre
cloud_per = 100# Maximo porcentaje de cobertura de nubes a considerar
composite = 'ndvi' # Funcion para generar el composite (median, ndvi)

compos20 = composite_l8_vegetation(init_date=range_date[0],
                                      last_date=range_date[1],
                                      roi=roi,
                                      cloud_per=cloud_per,
                                      calendar=calendar,
                                      composite='ndvi')

MJK = compos.select("NDVI")

NNDVI = compos.select('NDVI')
NNDVI2 = compos20.select('NDVI')

ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00"]
paleta = ["#FF0000"]
pale =["#1E8449"]

#"#FFF9e7","#F9E79F","#F7DC6F","F5B041","E67E22",
#f7dc6f
#f5b041
#e67e22
#cb4335]
resta = NNDVI.subtract(NNDVI2)

restamas = resta.gte(0.01)

restak = resta.updateMask(restamas)

   #mapdisplay(centroid, mydicc, Tiles="OpensTreetMap",zoom_start=9)
l78_viz_params84 = {'bands':['RED','GREEN','BLUE'],'min':0,'max': 2000, 'gamma':1.2}
l78_viz_params2020 = {'bands':['RED','GREEN','BLUE'],'min':0,'max': 2000, 'gamma':1.2}
    #l78_viz_params = {'bands':['NDVI'],'min':0,'max': 6000, 'gamma':1.2,}
mydicc = {'composite_84':compos.getMapId(l78_viz_params84),\
          'composite_2020':compos20.getMapId(l78_viz_params2020)
          }
mapdisplay(center,mydicc, zoom_start= 8)


In [ ]:
mydicc = {'composite_84':compos.getMapId(l78_viz_params84),\
          'composite_2020':compos20.getMapId(l78_viz_params2020),\
          'ndvi_DESPUES': NNDVI2.getMapId({'min':-0.1,'max':1,'palette':ndvi_palette}),\
          'ndvi_ANTES': NNDVI.getMapId({'min':-0.1,'max':1,'palette':ndvi_palette}),\
          'resta': restak.getMapId({'min':-0.1,'max':0.9,'palette': paleta})
          }
mapdisplay(center,mydicc, zoom_start= 8)
